In [1]:
!pip install scapy

     |████████████████████████████████| 1.0MB 9.0MB/s 
  Created wheel for scapy: filename=scapy-2.4.4-py2.py3-none-any.whl size=1189182 sha256=c6013873948fc1005133798b268966d7833c851c998d0ed93961bb0b1dbd1540
  Stored in directory: /root/.cache/pip/wheels/2c/e7/01/f097df99ac9cd0d4f744c255f918d471d7a4b0766bc84c38c3
Successfully built scapy


In [2]:
import os

In [3]:
os.chdir('/content/drive/MyDrive/stageINRIA')

In [4]:
import numpy as np
import glob

In [5]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms

In [6]:
from data_extraction import DataGenerator2

In [7]:
pth='./preprocessed'

In [8]:
###############Uncomment only if you didn't divide the dataset beetwen trainign and testing set#############################################
#import glob
#list_pkt=glob.glob(pth+'/*/*.bin')
#os.mkdir(pth+'/train')
#os.mkdir(pth+'/test')

In [9]:
###############Uncomment only if you didn't divide the dataset beetwen trainign and testing set############################################
#from sklearn.model_selection import train_test_split
#train, test = train_test_split(list_pkt, shuffle=True)

In [10]:
###############Uncomment only if you didn't divide the dataset beetwen trainign and testing set#############################################
#from shutil import copyfile
#for file_name in train:
#    filename=os.path.split(file_name)[-1]
#    filename=pth+'/train/'+filename
#    copyfile(file_name, filename)
#for file_name in test:
#    filename=os.path.split(file_name)[-1]
#    filename=pth+'/test/'+filename
#    copyfile(file_name, filename)

In [11]:
img_shape = (28,28)

In [12]:
train_batch_size=64
validation_batch_size=512

In [13]:
training_data=glob.glob(pth+'/train/*.bin')
validation_data=glob.glob(pth+'/test/*.bin')

In [14]:
training_generator=DataGenerator2(training_data, img_shape, 2, 16, train_batch_size)
validation_generator=DataGenerator2(validation_data, img_shape, 2, 16, validation_batch_size)

In [15]:
class Simple_autoencoder(nn.Module):
    def __init__(self, input_shape, latent_dim):
        super(Simple_autoencoder, self).__init__()
        self.input_shape = input_shape
        self.d=latent_dim
        self.encoder = nn.Sequential(
            nn.Linear(np.prod(input_shape),128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Linear(64, 12),
            nn.ReLU(True),
            nn.Linear(12, latent_dim))
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Linear(128, np.prod(input_shape)),
            nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [16]:
num_epochs = 100
learning_rate = 1e-3

In [17]:
model=Simple_autoencoder(img_shape, 3).cuda()
criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [18]:
def train(model, training_data, validation_data, criterion, optimizer, num_epochs, conv=False):  
    for epoch in range(num_epochs):
        train_loss=0
        model.train()
        # ====================training=====================
        for data in training_data:
            img, _ = data
            img = torch.from_numpy(img/255).float()
            if not conv:
                img = img.view(img.size(0), -1)
            else:
                img=img.squeeze(1)
            img = Variable(img).cuda()
            # ===================forward=====================
            output = model(img)
            loss = criterion(output, img)
            train_loss += loss
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # ====================validation=====================
        validation_loss=0
        model.eval()
        for data in validation_data:
            img, _ = data
            img = torch.from_numpy(img/255).float()
            if not conv:
                img = img.view(img.size(0), -1)
            else:
                img=img.unsqueeze(1)
            img = Variable(img).cuda()
            # ===================forward=====================
            output = model(img)
            validation_loss += criterion(output, img)
        # ===================log========================
        train_loss=train_loss / len(training_generator)
        validation_loss=validation_loss / len(validation_generator)
    
        print('epoch [{}/{}], train_loss:{:.4f}, validation_loss:{:.4f}'
              .format(epoch + 1, num_epochs, train_loss.data.item(), validation_loss.data.item()))
    return model

In [19]:
#train(model, training_generator, validation_generator, criterion, optimizer, num_epochs)

In [20]:
class Variationnal_autoencoder(nn.Module):
    def __init__(self, input_shape, latent_dim):
        super(Variationnal_autoencoder, self).__init__()
        self.input_shape = input_shape
        self.d = latent_dim
        
        self.fc_in = nn.Linear(np.prod(input_shape), 128)
        self.fc1 = nn.Linear(128,64)
        self.fc21 = nn.Linear(64, latent_dim)
        self.fc22 = nn.Linear(64, latent_dim)
        self.fc3 = nn.Linear(latent_dim, 64)
        self.fc4 = nn.Linear(64, 128)
        self.fc_out = nn.Linear(128, np.prod(input_shape))

    def encode(self, x):
        h1 = nn.functional.relu(self.fc_in(x))
        h2 = nn.functional.relu(self.fc1(h1))
        return self.fc21(h2), self.fc22(h2) 
    
    def decode(self, z):
        h3 = nn.functional.relu(self.fc3(z))
        h4 = nn.functional.relu(self.fc4(h3))
        return torch.sigmoid(self.fc_out(h4))

    def reparameterise(self, m, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = std.data.new(std.size()).normal_()
            return eps.mul(std).add_(m)
        else:
            return m

    def forward(self, x):
        moyenne, log_variance = self.encode(x)
        z = self.reparameterise(moyenne, log_variance)
        result = [self.decode(z), moyenne, log_variance]
        return result

In [21]:
reconstruction_function = nn.MSELoss(reduction='sum')

In [22]:
def custom_loss(output, x):

    RL = reconstruction_function(output[0], x)  # mse loss
    KLD_element = output[1].pow(2).add_(output[2].exp()).mul_(-1).add_(1).add_(output[2])
    KLD = torch.sum(KLD_element).mul_(-0.5)
    return RL + KLD


In [23]:
model=Variationnal_autoencoder(img_shape, 20).cuda()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [24]:
#train(model, training_generator, validation_generator, custom_loss, optimizer, num_epochs)

In [25]:
class Conv_Variationnal_autoencoer(nn.Module):
    def __init__(self, input_shape, latent_dim):
        super(Conv_Variationnal_autoencoer, self).__init__()
        self.input_shape = input_shape
        self.nz = latent_dim

        self.encoder = nn.Sequential(
            # input is 28 x 28
            nn.Conv2d(1, 32, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. 32 x 14 x 14
            nn.Conv2d(32, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. 64 x 7 x 7
            nn.Conv2d(64, 128, 3, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. 128 x 4 x 4
            nn.Conv2d(128, 256, 4, 1, 0, bias=False),
            # nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            # nn.Sigmoid()
        )

        self.decoder = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     256, 128, 4, 1, 0, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            # state size. 128 x 4 x 4
            nn.ConvTranspose2d(128, 64, 3, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # state size. 64 x 8 x 8
            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            # state size. 32 x 16 x 16
            nn.ConvTranspose2d(32,     1, 4, 2, 1, bias=False),
            # nn.BatchNorm2d(ngf),
            # nn.ReLU(True),
            # state size. 32 x 32 x 32
            # nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            # nn.Tanh()
            nn.Sigmoid()
            # state size. (nc) x 64 x 64
        )

        self.fc1 = nn.Linear(256, 128)
        self.fc_mu = nn.Linear(128, latent_dim)
        self.fc_var = nn.Linear(128, latent_dim)

        self.fc3 = nn.Linear(latent_dim, 128)
        self.fc4 = nn.Linear(128, 256)


    def encode(self, x):
        conv = self.encoder(x);
        #print("encode conv", conv.size())
        h1 = self.fc1(conv.view(-1, 256))
        #print("encode h1", h1.size())
        return self.fc_mu(h1), self.fc_var(h1)

    def decode(self, z):
        h3 = nn.functional.relu(self.fc3(z))
        deconv_input = self.fc4(h3)
        #print("deconv_input", deconv_input.size())
        deconv_input = deconv_input.view(-1,256,1,1)
        #print("deconv_input", deconv_input.size())
        return self.decoder(deconv_input)

    def reparametrize(self, m, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = std.data.new(std.size()).normal_()
            return eps.mul(std).add_(m)
        else:
            return m

    def forward(self, x):
        #print("x", x.size())
        moyenne, log_variance = self.encode(x)
        #print("moyenne, log_variance", moyenne.size(), log_variance.size())
        z = self.reparametrize(moyenne, log_variance)
        #print("z", z.size())
        result = [self.decode(z), moyenne, log_variance]
        #print("decoded", self.decode(z).size())
        return result

In [26]:
model=Conv_Variationnal_autoencoer(img_shape, 20).cuda()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [27]:
#train(model, training_generator, validation_generator, custom_loss, optimizer, num_epochs, conv=True)

In [28]:
def BVAE_loss(output, x, B, hyperparameters):
    RL = reconstruction_function(output[0], x)  # mse loss
    KLD_element = output[1].pow(2).add_(output[2].exp()).mul_(-1).add_(1).add_(output[2])
    KLD = torch.sum(KLD_element).mul_(-0.5)
    if hyperparameters == None: # https://openreview.net/forum?id=Sy2fzU9gl
        loss = RL + B * KLD
    elif len(hyperparameters)==3: # https://arxiv.org/pdf/1804.03599.pdf
        self.C_max = self.torch.Tensor([hyperparameters[1]])
        C = torch.clamp(self.C_max/self.C_stop_iter * self.num_iter, 0, self.C_max.data[0])
        #loss = recons_loss + self.gamma * kld_weight* (kld_loss - C).abs()
    else:
        raise ValueError('Undefined loss type.')
    return loss

In [29]:
def custom_loss(output, x):
    return(BVAE_loss(output, x, 5, None))

In [30]:
model=Conv_Variationnal_autoencoer(img_shape, 20).cuda()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [31]:
#train(model, training_generator, validation_generator, custom_loss, optimizer, num_epochs, conv=True)

In [32]:
from models.beta_vae import BetaVAE

In [33]:
def training_step(self, batch, batch_size, len_training_set):
        real_img, labels = batch
        real_img = torch.from_numpy(real_img/255).float()
        real_img=real_img.unsqueeze(1)
        real_img=Variable(real_img).cuda()
        results = self.forward(real_img, labels = labels)
        train_loss = self.loss_function(*results,
                                              M_N = batch_size/len_training_set,
                                              )
        
        return train_loss

BetaVAE.training_step=training_step

In [34]:
latent_dim=20
hidden_dims=[32, 64, 128]
loss_type='B'

In [35]:
model=BetaVAE(in_channels = 1, latent_dim=latent_dim, hidden_dims=hidden_dims, loss_type=loss_type).cuda()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [36]:
def train(model, training_data, validation_data, optimizer, num_epochs):  
    for epoch in range(num_epochs):
        train_loss=0
        model.train()
        # ====================training=====================
        for data in training_data:
            # ===================forward====================
            loss = model.training_step(data, len(data[0]), 1/len(training_data))
            train_loss+=loss['loss']
            # ===================backward====================
            optimizer.zero_grad()
            loss['loss'].backward()
            optimizer.step()
        # ====================validation=====================
        validation_loss=0
        model.eval()
        for data in validation_data:
            # ===================forward=====================
            loss = model.training_step(data, len(data[0]), 1/len(validation_data))
            validation_loss+=loss['loss']
        # ===================log========================
        train_loss=train_loss / len(training_generator)
        validation_loss=validation_loss / len(validation_generator)
    
        print('epoch [{}/{}], train_loss:{:.4f}, validation_loss:{:.4f}'
              .format(epoch + 1, num_epochs, train_loss.data.item(), validation_loss.data.item()))
    return model

In [37]:
#train(model, training_generator, validation_generator, optimizer, num_epochs)

In [ ]:
from models.vq_vae import VQVAE

VQVAE.training_step=training_step

embedding_dim=20
num_embeddings=5
hidden_dims=[32, 64, 128]
img_size=img_shape[0]

#model=VQVAE(in_channels = 1, embedding_dim=embedding_dim, hidden_dims=hidden_dims, num_embeddings=num_embeddings, img_size=img_size).cuda()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

train(model, training_generator, validation_generator, optimizer, num_epochs)

In [ ]:
x=validation_generator[0][0]
x=torch.from_numpy(x/255).float()
x=x.unsqueeze(1)
x=Variable(x).cuda()

In [ ]:
x_hat=model.generate(x)

In [ ]:
from data_extraction import convert_image_to_sequ
def binary_error(x_hat,x):
    x_hat=torch.squeeze(x_hat)
    x=torch.squeeze(x)
    x=x.cpu().numpy()
    x=x*255
    x_hat=x_hat.cpu().detach().numpy()
    x_hat=x_hat*255
    r=0
    for idx, img in enumerate(x_hat):
        output =convert_image_to_sequ(img)
        target =convert_image_to_sequ(x[idx])
        sum = np.asarray(output)==np.asarray(target)
        r+=len(sum) - np.count_nonzero(sum)
    return r


In [ ]:
error = binary_error(x_hat, x)
print(1-error/(250*28*28))

In [ ]:
x_g=model.sample(1,0)

In [ ]:
x_g=torch.squeeze(x_g)
x_g = torch.nn.functional.relu(x_g)
x_g=x_g.cpu().detach().numpy()
x_g=x_g*255

In [ ]:
sequ = convert_image_to_sequ(x_g)
print(sequ)

In [ ]:
from utils import hexli_to_packet
pkt=hexli_to_packet(sequ)
print(pkt)

In [ ]:
from scapy.all import send, IP, Ether, UDP, TCP, ICMP, Ether
pkt=Ether(pkt)
pkt

In [ ]:
def test_generation(pkt, level=4):
    if level == 4:
        if any(x in pkt for x in [UDP, TCP, ICMP]):
            return True
    if level == 3:
        if IP in pkt:
            return True
    if level == 2:
        if Ether in pkt:
            return True

In [ ]:
print(test_generation(pkt, level=3))

In [ ]:
def evaluate_generation(model, N, level = 4):
    x_g=model.sample(N,0)
    x_g=torch.squeeze(x_g)
    x_g = torch.nn.functional.relu(x_g)
    x_g=x_g.cpu().detach().numpy()
    x_g=x_g*255
    score=0
    for x in x_g:
        sequ = convert_image_to_sequ(x)
        pkt = hexli_to_packet(sequ)
        pkt = Ether(pkt)
        if test_generation(pkt, level):
            score+=1
    return score/N

In [ ]:
s = evaluate_generation(model, 1000, 4)
print(s)

In [ ]:
pkt.show()

In [ ]:
pkt[TCP].fields

In [ ]:
import pandas as pd

def pkt_to_csv(pkt):
    d={}
    for i, layer in enumerate(pkt.layers()):
        field_names = [field.name for field in pkt[layer].fields_desc]
        fields = {'l{}.{}'.format(i+1, field_name): [getattr(pkt[layer], field_name)] for field_name in field_names}
        d.update(fields)
    return pd.DataFrame.from_dict(d)

In [ ]:
print(pkt_to_csv(pkt))

In [ ]:
def generate_pkts(model, N):
    lst=[]
    x_g=model.sample(N,0)
    x_g=torch.squeeze(x_g)
    x_g = torch.nn.functional.relu(x_g)
    x_g=x_g.cpu().detach().numpy()
    x_g=x_g*255
    score=0
    for x in x_g:
        sequ = convert_image_to_sequ(x)
        pkt = hexli_to_packet(sequ)
        pkt = Ether(pkt)
        lst.append(pkt)
    return lst

In [ ]:
pkts= generate_pkts(model, 1000)

In [ ]:
csv_g=pkt_to_csv(pkts[0])
for pkt in pkts[1:]:
    csv_g=csv_g.append(pkt_to_csv(pkt))
csv_g['Label']=False
print(csv_g.shape)

In [ ]:
import re
pkts=[]
ratio=int(np.prod(img_shape)/8)
for path in training_data+validation_data:
    sample=open(path, 'rb').read()
    li=sample.hex()
    li=re.findall('..',li)
    li=li[:ratio]
    li=li+['00']*(ratio-len(li))
    pkt=hexli_to_packet(li)
    pkt=Ether(pkt)
    pkts.append(pkt)


In [ ]:
csv_t=pkt_to_csv(pkts[0])
for pkt in pkts[1:]:
    csv_t=csv_t.append(pkt_to_csv(pkt))
csv_t['Label']=True
print(csv_t.shape)

In [ ]:
csv=csv_g.append(csv_t)
print(csv.shape)

In [ ]:
csv[csv['Label']==False]